In [1]:
import sys
import os
import pandas as pd
import numpy as np
from itertools import product

from sklearn.metrics import root_mean_squared_error

import matplotlib.pyplot as plt
from skforecast.plot import set_dark_theme
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as poff
pio.templates.default = "seaborn"
poff.init_notebook_mode(connected=True)

os.environ["KERAS_BACKEND"] = "tensorflow" # 'tensorflow', 'jax´ or 'torch'
import keras
from keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from keras.callbacks import EarlyStopping

if keras.__version__ > "3.0":
    if keras.backend.backend() == "tensorflow":
        import tensorflow
    elif keras.backend.backend() == "torch":
        import torch
    else:
        print("Backend not recognized. Please use 'tensorflow' or 'torch'.")

import skforecast
from skforecast.deep_learning import ForecasterRnn
from skforecast.deep_learning.utils import create_and_compile_model
from sklearn.preprocessing import MinMaxScaler
from skforecast.model_selection import TimeSeriesFold
from skforecast.model_selection import backtesting_forecaster_multiseries

import warnings
warnings.filterwarnings('once')

In [2]:
def load_datasets():
    current_dir = os.getcwd()
    ROOT_PATH = os.path.dirname(current_dir)
    sys.path.insert(1, ROOT_PATH)
    import root

    train = pd.read_pickle(root.DIR_DATA_STAGE + 'train.pkl')
    test = pd.read_pickle(root.DIR_DATA_STAGE + 'test.pkl')
    return root, train, test

In [3]:
def create_model(data_train, levels, lags, steps, recurrent_units, dense_units, learning_rate):
    model = create_and_compile_model(
        series=data_train,
        levels=levels, 
        lags=lags,
        steps=steps,
        recurrent_layer="LSTM",
        recurrent_units=recurrent_units,
        dense_units=dense_units,
        optimizer=Adam(learning_rate=learning_rate), 
        loss=MeanSquaredError()
    )
    return model

In [4]:
def create_forecaster(data_train, data_val, model, levels, steps, lags, epochs, batch_size):
    forecaster = ForecasterRnn(
        regressor=model,
        levels=levels,
        steps=steps,
        lags=lags,
        transformer_series=MinMaxScaler(),
        fit_kwargs={
            "epochs": epochs,             # Número de épocas para entrenar el modelo.
            "batch_size": batch_size,     # Tamaño del batch para entrenar el modelo.
            "series_val": data_val,       # Datos de validación para el entrenamiento del modelo.
        },
    )
    forecaster.fit(data_train)
    return forecaster

In [5]:
def backtesting(data, end_val, forecaster, levels):
    cv = TimeSeriesFold(
        steps=forecaster.max_step,
        initial_train_size=len(data.loc[:end_val, :]),
        refit=False,
    )
    metrics, predictions = backtesting_forecaster_multiseries(
        forecaster=forecaster,
        series=data,
        levels=forecaster.levels,
        cv=cv,
        metric=root_mean_squared_error,
        verbose=False,
    )
    return metrics, predictions

In [6]:
root, train, test = load_datasets()
data = pd.concat([train, test])
end_val = '2022-08-31 23:59:59'
val = train.loc[end_val:]
train = train.loc[:end_val]

In [7]:
series = ['target', 'temperature', 'rain', 'snowfall', 'surface_pressure', 'cloudcover_total', 'windspeed_10m', 'winddirection_10m', 'shortwave_radiation', 'euros_per_mwh', 'installed_capacity'] 
levels = ['target']  # Serie que se quiere predecir
steps = 24           # Pasos a futuro a predecir

data_train = train[series].copy()
data_val = val[series].copy()
data_test = test[series].copy()

In [8]:
param_grid = {
    "lags": [24, 72],
    "recurrent_units": [[100, 50], [128, 64]],
    "dense_units": [[32, 16], [64, 32]],
    "learning_rate": [0.01, 0.001],
    "epochs": [4, 8],
    "batch_size": [64, 128],
}

grid = list(product(*param_grid.values()))
best_metrics = float("inf")
best_params = None
best_forecaster = None
best_predictions = None

In [10]:
#metrics

In [11]:
results = pd.DataFrame({
    'lags': [],
    'recurrent_units': [],
    'dense_units': [],
    'learning_rate': [],
    'epochs': [],
    'batch_size': [],
    'RMSE': []
    })
for i, params in enumerate(grid):
    print(f"Training model {i + 1}/{len(grid)} with params: {params}")
    lags, recurrent_units, dense_units, learning_rate, epochs, batch_size = params

    # Create the model
    model = create_model(data_train, levels, lags, steps, recurrent_units, dense_units, learning_rate)

    # Create the forecaster
    forecaster = create_forecaster(data_train, data_val, model, levels, steps, lags, epochs, batch_size)

    # Evaluate performance using backtesting
    metrics, predictions = backtesting(data, end_val, forecaster, levels)

    # Check if this combination is the best so far
    current_metric = metrics.root_mean_squared_error.values[0]
    print(current_metric, type(current_metric), best_metrics, type(best_metrics))
    if current_metric < best_metrics:
        best_metrics = current_metric
        best_params = params
        best_forecaster = forecaster
        best_predictions = predictions
        print(f"New best model found with RMSE: {current_metric}")
    
    # Save results
    new_row = pd.DataFrame([{
        'recurrent_units': recurrent_units,
        'dense_units': dense_units,
        'learning_rate': learning_rate,
        'epochs': epochs,
        'batch_size': batch_size,
        'RMSE': current_metric
    }])
    results = pd.concat([results, new_row], ignore_index=True)

print(f"\nBest hyperparameters: {best_params}")
print(f"Best validation RMSE: {best_metrics}")
results.to_excel(root.DIR_DATA_ANALYTICS + 'LSTM_grid_search.xlsx', index=False)

Training model 1/64 with params: (24, [100, 50], [32, 16], 0.01, 4, 64)
keras version: 3.6.0
Using backend: tensorflow
tensorflow version: 2.18.0
Epoch 1/4
136/136 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0295 - val_loss: 0.0093
Epoch 2/4
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0131 - val_loss: 0.0224
Epoch 3/4
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0114 - val_loss: 0.0173
Epoch 4/4
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0106 - val_loss: 0.0359
Epoch 1/4
136/136 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0099 - val_loss: 0.0250
Epoch 2/4
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0088 - val_loss: 0.0168
Epoch 3/4
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0079 - val_loss: 0.0555
Epoch 4/4
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0109 - val_loss: 0.0112


  0%|          | 0/273 [00:00<?, ?it/s]

1032.427409860232 <class 'numpy.float64'> inf <class 'float'>
New best model found with RMSE: 1032.427409860232
Training model 2/64 with params: (24, [100, 50], [32, 16], 0.01, 4, 128)
keras version: 3.6.0
Using backend: tensorflow
tensorflow version: 2.18.0
Epoch 1/4
68/68 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 0.0305 - val_loss: 0.0071
Epoch 2/4
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0115 - val_loss: 0.0115
Epoch 3/4
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0096 - val_loss: 0.0098
Epoch 4/4
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0088 - val_loss: 0.0131
Epoch 1/4
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 0.0086 - val_loss: 0.0118
Epoch 2/4
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0080 - val_loss: 0.0134
Epoch 3/4
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0074 - val_loss: 0.0130
Epoch 4/4
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0073 - val_loss: 0.0112


  0%|          | 0/273 [00:00<?, ?it/s]

1307.9098493723786 <class 'numpy.float64'> 1032.427409860232 <class 'numpy.float64'>
Training model 3/64 with params: (24, [100, 50], [32, 16], 0.01, 8, 64)
keras version: 3.6.0
Using backend: tensorflow
tensorflow version: 2.18.0
Epoch 1/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0235 - val_loss: 0.0053
Epoch 2/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0130 - val_loss: 0.0244
Epoch 3/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0104 - val_loss: 0.0125
Epoch 4/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0083 - val_loss: 0.0058
Epoch 5/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0154 - val_loss: 0.0079
Epoch 6/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0087 - val_loss: 0.0146
Epoch 7/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0116 - val_loss: 0.0071
Epoch 8/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0082 - val_loss: 0.0121
Epoch 1/8
136/136 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: 0.0076 - val_loss: 0.

...

  0%|          | 0/273 [00:00<?, ?it/s]

7143705.561761599 <class 'numpy.float64'> 925.9732044689384 <class 'numpy.float64'>
Training model 64/64 with params: (72, [128, 64], [64, 32], 0.001, 8, 128)
keras version: 3.6.0
Using backend: tensorflow
tensorflow version: 2.18.0
Epoch 1/8
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 163ms/step - loss: 0.0320 - val_loss: 0.0125
Epoch 2/8
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 159ms/step - loss: 0.0154 - val_loss: 0.0104
Epoch 3/8
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 160ms/step - loss: 0.0102 - val_loss: 0.0106
Epoch 4/8
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 160ms/step - loss: 0.0088 - val_loss: 0.0218
Epoch 5/8
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 160ms/step - loss: 0.0082 - val_loss: 0.0149
Epoch 6/8
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 161ms/step - loss: 0.0072 - val_loss: 0.0234
Epoch 7/8
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 162ms/step - loss: 0.0072 - val_loss: 291.5618
Epoch 8/8
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 161ms/step - loss: 0.0063 - val_loss: 50238200.0000
Epoch 1/8
68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 166ms/step - loss: 0.0052 - v

  0%|          | 0/273 [00:00<?, ?it/s]

1390661.627878348 <class 'numpy.float64'> 925.9732044689384 <class 'numpy.float64'>

Best hyperparameters: (72, [128, 64], [32, 16], 0.01, 4, 64)
Best validation RMSE: 925.9732044689384


ModuleNotFoundError: No module named 'openpyxl'

In [12]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


c:\Users\jgome\anaconda3\envs\time_series\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning:

unclosed file <_io.BufferedWriter name=3>

c:\Users\jgome\anaconda3\envs\time_series\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning:

unclosed file <_io.BufferedReader name=4>

c:\Users\jgome\anaconda3\envs\time_series\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning:

unclosed file <_io.BufferedReader name=5>



In [13]:
results.to_excel(root.DIR_DATA_ANALYTICS + 'LSTM_grid_search.xlsx', index=False)